In [1]:
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import datetime as dt
import time

In [2]:
# portfolio of crypto
crypto = ['BNB-USD', 'ETH-USD', 'BTC-USD', 'DOGE-USD', 'LINK-USD', 'SOL-USD', 'ADA-USD', 'XRP-USD', 'LTC-USD']

#### Download historical data from yf API

In [3]:
import yfinance as yf

start_date = "2020-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(crypto, start=start_date, end=end_date, group_by='ticker')

# download data to csv
data.to_csv("crypto.csv")

[*********************100%***********************]  9 of 9 completed


#### Download analysis from csv downloaded data

In [4]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("crypto.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Forward fill missing values for each column
df = df.fillna(method='ffill')

# Convert all values to float
df = df.astype(float)

# # rename tickers to more readable names
# df.columns = pd.MultiIndex.from_tuples([
#     (
#         col[0].replace('BNB-USD', 'BNB').replace('ETH-USD', 'ETH').replace('BTC-USD', 'BTC').replace('DOGE-USD', 'DOGE').replace('LINK-USD', 'LINK').replace('SOL-USD', 'SOL'),
#         col[1]
#     ) for col in df.columns
# ])


# Show the result
df.head()

Ticker          BTC-USD                                                       \
Price              Open         High          Low        Close        Volume   
Date                                                                           
2020-01-01  7194.892090  7254.330566  7174.944336  7200.174316  1.856566e+10   
2020-01-02  7202.551270  7212.155273  6935.270020  6985.470215  2.080208e+10   
2020-01-03  6984.428711  7413.715332  6914.996094  7344.884277  2.811148e+10   
2020-01-04  7345.375488  7427.385742  7309.514160  7410.656738  1.844427e+10   
2020-01-05  7410.451660  7544.497070  7400.535645  7411.317383  1.972507e+10   

Ticker         ETH-USD                                                    ...  \
Price             Open        High         Low       Close        Volume  ...   
Date                                                                      ...   
2020-01-01  129.630661  132.835358  129.198288  130.802002  7.935230e+09  ...   
2020-01-02  130.820038  130.820038  126.954910  127.410179  8.032709e+09  ...   
2020-01-03  127.411263  134.554016  126.490021  134.171707  1.047685e+10  ...   
2020-01-04  134.168518  136.052719  133.040558  135.069366  7.430905e+09  ...   
2020-01-05  135.072098  139.410202  135.045624  136.276779  7.526675e+09  ...   

Ticker        LTC-USD                                                 \
Price            Open       High        Low      Close        Volume   
Date                                                                   
2020-01-01  41.326534  42.285019  41.288033  42.017719  2.782992e+09   
2020-01-02  42.018085  42.134747  39.683567  39.823013  2.759827e+09   
2020-01-03  39.863129  42.447174  39.450844  42.415573  3.260961e+09   
2020-01-04  42.383526  43.342705  41.933075  43.326607  2.843193e+09   
2020-01-05  43.291382  44.733616  42.998535  43.553207  3.017148e+09   

Ticker       XRP-USD                                              
Price           Open      High       Low     Close        Volume  
Date                                                              
2020-01-01  0.192912  0.194362  0.192107  0.192667  1.041134e+09  
2020-01-02  0.192708  0.192896  0.186947  0.188043  1.085351e+09  
2020-01-03  0.187948  0.194070  0.185846  0.193521  1.270017e+09  
2020-01-04  0.193521  0.194653  0.191835  0.194355  9.993316e+08  
2020-01-05  0.194367  0.199223  0.193884  0.195537  1.168068e+09  

[5 rows x 45 columns]

In [5]:
# # Here you can plot the charts for every ticker

# # For every ticker, plot Close
# import matplotlib.pyplot as plt

# for ticker in df.columns.levels[0]:
#     plt.figure(figsize=(12, 6))
#     plt.plot(df[ticker]['Close'], label=ticker)
#     plt.title(f"{ticker} Close Price")
#     plt.xlabel("Date")
#     plt.ylabel("Close Price")
#     plt.legend()
#     plt.show()

#### Plot of Volume Profile for All crypto

In [6]:
import plotly.io as pio
pio.renderers.default = 'vscode'
import plotly.express as px
import plotly.graph_objects as go

for c in crypto:
    # Use the crypto data from our df
    df_crypto = df[c][['Close', 'Volume']]

    # Convert the crypto to datetime if needed
    if not isinstance(df_crypto.index, pd.DatetimeIndex):
        df_crypto.index = pd.to_datetime(df_crypto.index)

    # plot
    hist = go.Histogram(
        y=df_crypto['Close'],
        x=df_crypto['Volume'],
        nbinsy=150,
        orientation='h',
        name='Volume Profile',
        marker_color='#B0C4DE',
        opacity=0.5,
        xaxis='x2',
    )

    line = go.Scatter(
        x=df_crypto.index,
        y=df_crypto['Close'],
        mode='lines',
        name='Close',
        marker_color='blue',
        xaxis='x',
        yaxis='y',
    )

    fig = go.Figure([hist, line])

    fig.update_layout(
        xaxis=dict(
            title='Date',
            anchor='y',
            domain=[0, 1],
            showgrid=False,
            side='bottom'
        ),
        xaxis2=dict(
            title='Sum of Volume',
            overlaying='x',
            side='top',
            showgrid=False
        ),
        yaxis=dict(
            title='Close Price',
            showgrid=True
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        height=600,
        title=f'Volume Profile & {c} Close Prices'
    )

    fig.show()

In [7]:
# !pip install -U kaleido

# kaleido library is used for static image export of Plotly figures, 
# required for saving figures as images

In [8]:
# # Install Chrome for Kaleido image export
# import kaleido
# kaleido.get_chrome_sync();

In [9]:
# # Same iteration for all crypto as previous cell, just added the image saving with kaleido
# import plotly.io as pio
# pio.renderers.default = 'vscode'
# import plotly.express as px
# import plotly.graph_objects as go

# crypto_ = ['BNB-USD', 'ETH-USD', 'BTC-USD', 'DOGE-USD', 'LINK-USD', 'SOL-USD']

# for c in crypto_:
#     df_crypto = df[c][['Close', 'Volume']]

#     hist = go.Histogram(
#         y=df_crypto['Close'],
#         x=df_crypto['Volume'],
#         nbinsy=150,
#         orientation='h',
#         name='Volume Profile',
#         marker_color='#B0C4DE',
#         opacity=0.5,
#         xaxis='x2',
#     )

#     line = go.Scatter(
#         x=df_crypto.index,
#         y=df_crypto['Close'],
#         mode='lines',
#         name='Close',
#         marker_color='blue',
#         xaxis='x',
#         yaxis='y',
#     )

#     fig = go.Figure([hist, line])

#     fig.update_layout(
#         xaxis=dict(
#             title='Date',
#             anchor='y',
#             domain=[0, 1],
#             showgrid=False,
#             side='bottom'
#         ),
#         xaxis2=dict(
#             title='Sum of Volume',
#             overlaying='x',
#             side='top',
#             showgrid=False
#         ),
#         yaxis=dict(
#             title='Close Price',
#             showgrid=True
#         ),
#         legend=dict(
#             orientation='h',
#             yanchor='bottom',
#             y=1.02,
#             xanchor='right',
#             x=1
#         ),
#         height=600,
#         title=f'Volume Profile & {c} Close Prices'
#     )

#     fig.show()

#     # Salva il grafico come JPG
#     fig.write_image(f"{c}_volume_profile.jpg", format="jpg")